# В этом ноутбуке
я привожу данные в тестовом файле и в файле с данными, собранными для обучения, к единому виду:
- удаляю дубликаты, 
- заполняю или удаляю пропущенные значения
- оставляю одинаковое количество признаков

Файлы train_df_cleaned.csv и test_df_cleaned.csv готовы для дальнейшей предобработки.

In [222]:
import pandas as pd
pd.set_option("expand_frame_repr", True)
import numpy as np
import re
import time
from datetime import datetime
import json
from tqdm.notebook import tqdm
import pprint
pp=pprint.PrettyPrinter()

In [223]:
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [224]:
def get_unique_values(non_un_list):
    return list(set(non_un_list))
   

# Приведение данных к единому виду

### тестовый 

In [225]:
test_df=pd.read_csv('test.csv')

test_df['brand']=test_df['brand'].apply(lambda x: x.replace('MERCEDES', 'MERCEDES-BENZ')) ## НАДО НА БУДУЩЕЕ!

In [226]:
brands=get_unique_values(test_df['brand'])
display(brands)

['VOLKSWAGEN',
 'AUDI',
 'INFINITI',
 'LEXUS',
 'SKODA',
 'HONDA',
 'TOYOTA',
 'VOLVO',
 'MERCEDES-BENZ',
 'NISSAN',
 'BMW',
 'MITSUBISHI']

In [227]:
working_columns=[ 'brand', 'model_name','modelDate','bodyType','color','engineDisplacement', 
                 'numberOfDoors','vehicleTransmission', 'Привод', 'Руль', 
                 'enginePower','fuelType', 'productionDate', 'mileage',  
                 'Владельцы', 'ПТС'] #,'parsing_unixtime' Мы уже выяснили в предыдущем ноутбуке, что дата 21окт2020
test_df=test_df[working_columns]

In [228]:
#Переименуем колонки
columns_test=['brand','model','modeld','body','color','engine','doors',
              'transmiss','drive','steer','power','fuel','year','mileage','owners','title']
print(len(columns_test))
print(len(test_df.columns))
test_df.columns=columns_test
display(test_df.info())

16
16
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   brand      34686 non-null  object
 1   model      34686 non-null  object
 2   modeld     34686 non-null  int64 
 3   body       34686 non-null  object
 4   color      34686 non-null  object
 5   engine     34686 non-null  object
 6   doors      34686 non-null  int64 
 7   transmiss  34686 non-null  object
 8   drive      34686 non-null  object
 9   steer      34686 non-null  object
 10  power      34686 non-null  object
 11  fuel       34686 non-null  object
 12  year       34686 non-null  int64 
 13  mileage    34686 non-null  int64 
 14  owners     34686 non-null  object
 15  title      34685 non-null  object
dtypes: int64(4), object(12)
memory usage: 4.2+ MB


None

In [229]:
what=get_unique_values(test_df['title'])
print(what)
display(test_df['title'].value_counts())  # Количество оригиналов преобладает, заменим единственно пропущ значение на Оригинал
test_df.fillna('Оригинал',inplace=True)

['Оригинал', nan, 'Дубликат']


Оригинал    30098
Дубликат     4587
Name: title, dtype: int64

In [230]:
display(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   brand      34686 non-null  object
 1   model      34686 non-null  object
 2   modeld     34686 non-null  int64 
 3   body       34686 non-null  object
 4   color      34686 non-null  object
 5   engine     34686 non-null  object
 6   doors      34686 non-null  int64 
 7   transmiss  34686 non-null  object
 8   drive      34686 non-null  object
 9   steer      34686 non-null  object
 10  power      34686 non-null  object
 11  fuel       34686 non-null  object
 12  year       34686 non-null  int64 
 13  mileage    34686 non-null  int64 
 14  owners     34686 non-null  object
 15  title      34686 non-null  object
dtypes: int64(4), object(12)
memory usage: 4.2+ MB


None

In [231]:
display(test_df['fuel'].value_counts())

бензин     28601
дизель      5800
гибрид       223
электро       55
газ            7
Name: fuel, dtype: int64

### тренинговый

In [232]:
train_df=pd.read_csv('parsed_data.csv')
display(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41030 entries, 0 to 41029
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        41030 non-null  int64  
 1   brand        41030 non-null  object 
 2   0more        41030 non-null  float64
 3   1more        41030 non-null  object 
 4   2more        41028 non-null  object 
 5   3more        40876 non-null  object 
 6   model        41030 non-null  object 
 7   год выпуска  41030 non-null  float64
 8   Пробег       41030 non-null  object 
 9   Кузов        41028 non-null  object 
 10  Цвет         41030 non-null  object 
 11  Двигатель    41028 non-null  object 
 12  Налог        40044 non-null  object 
 13  Коробка      41028 non-null  object 
 14  Привод       41028 non-null  object 
 15  Руль         41030 non-null  object 
 16  Состояние    41030 non-null  object 
 17  Владельцы    41030 non-null  object 
 18  ПТС          41030 non-null  object 
 19  Влад

None

In [233]:
display(train_df['Состояние'].value_counts()) # Так же, как и в тестовом

Не требует ремонта    41030
Name: Состояние, dtype: int64

In [234]:
train_df.sample()

,index,brand,0more,1more,2more,3more,model,год выпуска,Пробег,Кузов,...,ПТС,Владение,Таможня,Гарантия,Обмен,VIN,Госномер,price,Кузов №,Запас хода
8007,8007,TOYOTA,2011.0,внедорожник 5 дв.,белый,Дизель,Toyota Land Cruiser Prado 150 Series,2011.0,220 000 км,внедорожник 5 дв.,...,Оригинал,NaN,Растаможен,NaN,Рассмотрю варианты,JTE**************,NaN,1885000,NaN,NaN


In [235]:
print(train_df.columns)

Index(['index', 'brand', '0more', '1more', '2more', '3more', 'model',
       'год выпуска', 'Пробег', 'Кузов', 'Цвет', 'Двигатель', 'Налог',
       'Коробка', 'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС',
       'Владение', 'Таможня', 'Гарантия', 'Обмен', 'VIN', 'Госномер', 'price',
       'Кузов №', 'Запас хода'],
      dtype='object')


In [236]:
# Колонки в тестовом сете:
columns_test=['brand','model','modeld','body','color','engine','doors',
              'transmiss','drive','steer','power','fuel','year','mileage','owners','title']

#Переименуем колонки а тренинговом:
columns_train=['todrop14','brand','todrop11','todrop12','todrop13','fuel','model',
               'year','mileage','body','color','engine','todrop1',
               'transmiss','drive','steer','todrop2','owners','title',
               'todrop3','todrop4','todrop5','todrop6','todrop7','todrop8',
               'price','todrop10','todrop9']

print(len(columns_train)) 
print(len(train_df.columns))

28
28


In [237]:
set(columns_test).intersection(set(columns_train))

{'body',
 'brand',
 'color',
 'drive',
 'engine',
 'fuel',
 'mileage',
 'model',
 'owners',
 'steer',
 'title',
 'transmiss',
 'year'}

In [238]:
missing_cols=set(columns_test).difference(set(columns_train))  # Каких колонок не хватает в тренинговом сете
print(missing_cols)

{'doors', 'modeld', 'power'}


In [239]:
set(columns_train).difference(set(columns_test)) # Колонки, которых нет в тестовом

{'price',
 'todrop1',
 'todrop10',
 'todrop11',
 'todrop12',
 'todrop13',
 'todrop14',
 'todrop2',
 'todrop3',
 'todrop4',
 'todrop5',
 'todrop6',
 'todrop7',
 'todrop8',
 'todrop9'}

In [240]:
train_df.columns=columns_train # Само, собственно, переименование

In [241]:
train_df.sample(1) # Получилось

,todrop14,brand,todrop11,todrop12,todrop13,fuel,model,year,mileage,body,...,title,todrop3,todrop4,todrop5,todrop6,todrop7,todrop8,price,todrop10,todrop9
19375,19378,MERCEDES-BENZ,2002.0,купе-хардтоп,серебристый,Бензин,Mercedes-Benz CLK-Класс II (W209) 200,2002.0,283 000 км,купе-хардтоп,...,Дубликат,NaN,Растаможен,NaN,Рассмотрю варианты,WDB**************,******,430000,NaN,NaN


In [242]:
for col in missing_cols:   #Добавление колонок
    train_df[col]=' '

In [243]:
train_df.sample(1)

,todrop14,brand,todrop11,todrop12,todrop13,fuel,model,year,mileage,body,...,todrop5,todrop6,todrop7,todrop8,price,todrop10,todrop9,doors,modeld,power
5099,5099,TOYOTA,2017.0,внедорожник 5 дв.,коричневый,Бензин,Toyota RAV4 IV (CA40) Рестайлинг,2017.0,74 183 км,внедорожник 5 дв.,...,NaN,Рассмотрю варианты,XW7**************,NaN,1695000,NaN,NaN,,,


In [244]:
# Выкинем всё лишнее

train_df=train_df[['brand','model','modeld','body','color','engine','doors',
              'transmiss','drive','steer','power','fuel','year','mileage','owners','title','price']].copy()

In [245]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41030 entries, 0 to 41029
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   brand      41030 non-null  object 
 1   model      41030 non-null  object 
 2   modeld     41030 non-null  object 
 3   body       41028 non-null  object 
 4   color      41030 non-null  object 
 5   engine     41028 non-null  object 
 6   doors      41030 non-null  object 
 7   transmiss  41028 non-null  object 
 8   drive      41028 non-null  object 
 9   steer      41030 non-null  object 
 10  power      41030 non-null  object 
 11  fuel       40876 non-null  object 
 12  year       41030 non-null  float64
 13  mileage    41030 non-null  object 
 14  owners     41030 non-null  object 
 15  title      41030 non-null  object 
 16  price      41030 non-null  object 
dtypes: float64(1), object(16)
memory usage: 5.3+ MB


In [246]:
display(train_df.sample(1))

,brand,model,modeld,body,color,engine,doors,transmiss,drive,steer,power,fuel,year,mileage,owners,title,price
25987,AUDI,Audi 80 V (B4),,седан,серебристый,1.6 л / 101 л.с. /,,механическая,передний,Левый,,Бензин,1994.0,300 000 км,3 или более,Оригинал,100000


In [247]:
display(test_df.sample(1))

,brand,model,modeld,body,color,engine,doors,transmiss,drive,steer,power,fuel,year,mileage,owners,title
7156,VOLVO,XC70,2013,универсал 5 дв.,чёрный,2.4 LTR,5,автоматическая,полный,Левый,181 N12,дизель,2014,130000,1 владелец,Оригинал


In [248]:
# Пропущенные значения  - кроме колонки fuel, там надо вставить опцию ЭЛЕКТРО

train_df.dropna(subset=['body', 'engine', 'transmiss', 'drive'], how='any', axis=0, inplace=True)
display(train_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41028 entries, 0 to 41029
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   brand      41028 non-null  object 
 1   model      41028 non-null  object 
 2   modeld     41028 non-null  object 
 3   body       41028 non-null  object 
 4   color      41028 non-null  object 
 5   engine     41028 non-null  object 
 6   doors      41028 non-null  object 
 7   transmiss  41028 non-null  object 
 8   drive      41028 non-null  object 
 9   steer      41028 non-null  object 
 10  power      41028 non-null  object 
 11  fuel       40876 non-null  object 
 12  year       41028 non-null  float64
 13  mileage    41028 non-null  object 
 14  owners     41028 non-null  object 
 15  title      41028 non-null  object 
 16  price      41028 non-null  object 
dtypes: float64(1), object(16)
memory usage: 5.6+ MB


None

In [249]:
original_train=pd.read_csv('parsed_data.csv') 

### model

In [250]:
test_models_dict={}
for brand in brands:
    test_models=get_unique_values(test_df[test_df['brand']==brand]['model'])
    test_models_dict.update({brand:sorted(test_models)})

pp.pprint(test_models_dict)

{'AUDI': ['100',
          '200',
          '80',
          '90',
          '920',
          'A1',
          'A2',
          'A3',
          'A4',
          'A4_ALLROAD',
          'A5',
          'A6',
          'A7',
          'A8',
          'ALLROAD',
          'COUPE',
          'E_TRON',
          'Q3',
          'Q5',
          'Q7',
          'Q8',
          'R8',
          'RS3',
          'RS4',
          'RS5',
          'RS6',
          'RS7',
          'RSQ3',
          'S3',
          'S4',
          'S5',
          'S6',
          'S7',
          'S8',
          'SQ5',
          'TT',
          'TTS',
          'TT_RS',
          'V8'],
 'BMW': ['02',
         '1ER',
         '2ACTIVETOURER',
         '2ER',
         '2GRANDTOURER',
         '321',
         '326',
         '340',
         '3ER',
         '4',
         '5ER',
         '6ER',
         '7ER',
         '8ER',
         'E3',
         'I3',
         'I8',
         'M1',
         'M2',
         'M3',
         '

In [251]:
train_models_dict={}
for brand in brands:
    train_models=get_unique_values(train_df[train_df['brand']==brand]['model'])
    train_models_dict.update({brand:sorted(train_models)})

pp.pprint(train_models_dict)

{'AUDI': ['Audi 100  IV (C4) S4',
          'Audi 100 II (C2)',
          'Audi 100 III (C3)',
          'Audi 100 III (C3) Рестайлинг',
          'Audi 100 IV (C4)',
          'Audi 200 I (C2)',
          'Audi 200 II (C3)',
          'Audi 200 II (C3) Рестайлинг',
          'Audi 80  III (B2) 5 speed',
          'Audi 80  IV (B3)',
          'Audi 80 III (B2)',
          'Audi 80 IV (B3)',
          'Audi 80 V (B4)',
          'Audi 90 I (B2)',
          'Audi 90 II (B3)',
          'Audi 920 I',
          'Audi A1 I (8X)',
          'Audi A1 Sportback I (8X)',
          'Audi A1 Sportback I (8X) Рестайлинг',
          'Audi A2',
          'Audi A3  III (8V) ultra',
          'Audi A3 I (8L)',
          'Audi A3 I (8L) Рестайлинг',
          'Audi A3 II (8P)',
          'Audi A3 II (8P) Рестайлинг 1',
          'Audi A3 II (8P) Рестайлинг 2',
          'Audi A3 III (8V)',
          'Audi A3 III (8V) Рестайлинг',
          'Audi A3 Sportback II (8P) Рестайлинг 1',
          'Audi A3 S

In [252]:
train_df['model']=train_df['model'].apply(lambda x:(str(x).split()[1].upper()).split('-')[0] if len((str(x).split()[1].upper()).split('-'))>1 else (str(x).split()[1].upper()))

In [253]:
test_df['model']=test_df['model'].apply(lambda x: str(x).split('_')[0] if len(str(x).split('_'))>1 else x)

In [254]:
test_models_dict={}
for brand in brands:
    test_models=get_unique_values(test_df[test_df['brand']==brand]['model'])
    test_models_dict.update({brand:sorted(test_models)})

pp.pprint(test_models_dict)

{'AUDI': ['100',
          '200',
          '80',
          '90',
          '920',
          'A1',
          'A2',
          'A3',
          'A4',
          'A5',
          'A6',
          'A7',
          'A8',
          'ALLROAD',
          'COUPE',
          'E',
          'Q3',
          'Q5',
          'Q7',
          'Q8',
          'R8',
          'RS3',
          'RS4',
          'RS5',
          'RS6',
          'RS7',
          'RSQ3',
          'S3',
          'S4',
          'S5',
          'S6',
          'S7',
          'S8',
          'SQ5',
          'TT',
          'TTS',
          'V8'],
 'BMW': ['02',
         '1ER',
         '2ACTIVETOURER',
         '2ER',
         '2GRANDTOURER',
         '321',
         '326',
         '340',
         '3ER',
         '4',
         '5ER',
         '6ER',
         '7ER',
         '8ER',
         'E3',
         'I3',
         'I8',
         'M1',
         'M2',
         'M3',
         'M4',
         'M5',
         'M6',
         'X1'

In [255]:
train_models_dict={}
for brand in brands:
    train_models=get_unique_values(train_df[train_df['brand']==brand]['model'])
    train_models_dict.update({brand:sorted(train_models)})

pp.pprint(train_models_dict)

{'AUDI': ['100',
          '200',
          '80',
          '90',
          '920',
          'A1',
          'A2',
          'A3',
          'A4',
          'A5',
          'A6',
          'A7',
          'A8',
          'COUPE',
          'E',
          'Q2',
          'Q3',
          'Q5',
          'Q7',
          'Q8',
          'R8',
          'RS',
          'S3',
          'S4',
          'S5',
          'S6',
          'S7',
          'S8',
          'SQ5',
          'SQ7',
          'SQ8',
          'TT',
          'TTS'],
 'BMW': ['02',
         '1',
         '2',
         '2000',
         '3',
         '321',
         '4',
         '5',
         '6',
         '600',
         '7',
         '8',
         'I3',
         'I8',
         'M2',
         'M3',
         'M4',
         'M5',
         'M6',
         'M8',
         'X1',
         'X2',
         'X3',
         'X4',
         'X5',
         'X6',
         'X7',
         'Z3',
         'Z4'],
 'HONDA': ['ACCORD',
         

### modeld  - to drop

In [256]:
# Не получилось надыбить данных ни откуда

### body

In [257]:
train_body=set(get_unique_values(train_df['body']))
print(train_body,len(train_body))

{'внедорожник 3 дв.', 'универсал 5 дв.', 'хэтчбек 5 дв.', 'лифтбек', 'фастбек', 'родстер', 'кабриолет', 'тарга', 'седан', 'пикап полуторная кабина', 'седан-хардтоп', 'компактвэн', 'купе', 'пикап одинарная кабина', 'фургон', 'купе-хардтоп', 'минивэн', 'внедорожник открытый', 'лимузин', 'седан 2 дв.', 'пикап двойная кабина', 'микровэн', 'хэтчбек 3 дв.', 'внедорожник 5 дв.'} 24


In [258]:
test_body=set(get_unique_values(test_df['body']))
print(test_body,len(test_body))

{'внедорожник 3 дв.', 'универсал 5 дв.', 'хэтчбек 5 дв.', 'лифтбек', 'фастбек', 'родстер', 'кабриолет', 'тарга', 'седан', 'пикап полуторная кабина', 'седан-хардтоп', 'компактвэн', 'купе', 'пикап одинарная кабина', 'фургон', 'купе-хардтоп', 'минивэн', 'внедорожник открытый', 'лимузин', 'седан 2 дв.', 'пикап двойная кабина', 'микровэн', 'хэтчбек 3 дв.', 'внедорожник 5 дв.'} 24


In [259]:
test_body.difference(train_body)   # Не требует корректировки

set()

### color

In [260]:
train_color=set(get_unique_values(train_df['color']))
print(len(train_color))

16


In [261]:
test_color=set(get_unique_values(test_df['color']))
print(len(test_color))

16


In [262]:
test_color.symmetric_difference(train_color)   # Не требует корректировки

set()

### engine & power & fuel

In [263]:
#  ТРЕНИНГОВЫЙ

In [264]:
train_engine=get_unique_values(train_df['engine'])  #EngineDisplacement = Объём двигателя
print(train_engine[980:989],'\n',len(train_engine))

['3.0 л / 313 л.с. /', '6.2 л / 571 л.с. /', '2.8 л / 197 л.с. /', '3.0 л / 163 л.с. /', '4.7 л / 288 л.с. /', '4.2 л / 125 л.с. /', '1.6 л / 101 л.с. /', '2.5 л / 179 л.с. /', '2.0 л / 194 л.с. /'] 
 989


In [265]:
# Для поля МОЩНОСТЬ заберём данные из поля ОБЪЁМ ДВИГАТЕЛЯ, учитывая, что там ещё попадаются ЭЛЕКТРОдвигатели

train_df.loc[:,'power']= train_df.loc[:,'engine'].apply(lambda x: int(x.split()[3]) if len(x.split())<7 else int(x.split()[0]))

In [266]:
train_power=get_unique_values(train_df['power']) # ЛОШАДИНЫЕ СИЛЫ
print(np.array(train_power).min(),len(train_power))

display(train_df['power'].value_counts(bins=5, sort=False)) #Что это за маленькие значения?

7 308


(6.367, 133.4]     8774
(133.4, 259.8]    24725
(259.8, 386.2]     5437
(386.2, 512.6]     1398
(512.6, 639.0]      694
Name: power, dtype: int64

In [267]:
train_df[train_df['power']<30]                                            # что за ерунда

,brand,model,modeld,body,color,engine,doors,transmiss,drive,steer,power,fuel,year,mileage,owners,title,price
7190,TOYOTA,COMS,,купе,белый,7 л.с. / 5 кВт / Электро,,автоматическая,задний,Правый,7,NaN,2013.0,700 км,1 владелец,Оригинал,280000
38539,BMW,600,,хэтчбек 3 дв.,бежевый,0.6 л / 19 л.с. /,,механическая,задний,Левый,19,Бензин,1959.0,80 000 км,3 или более,Оригинал,5000000


In [268]:
original_train.loc[7190]  # Это не ошибка, это антиквариат

index                              7190
brand                            TOYOTA
0more                              2013
1more                              купе
2more                             белый
3more                               NaN
model                       Toyota COMS
год выпуска                        2013
Пробег                           700 км
Кузов                              купе
Цвет                              белый
Двигатель      7 л.с. / 5 кВт / Электро
Налог                         0 ₽ / год
Коробка                  автоматическая
Привод                           задний
Руль                             Правый
Состояние            Не требует ремонта
Владельцы                    1 владелец
ПТС                            Оригинал
Владение                            NaN
Таможня                      Растаможен
Гарантия                            NaN
Обмен                               NaN
VIN                   TAK30-***********
Госномер                         ******


In [269]:
original_train.loc[38639]  # Это не ошибка, это почти игрушечная машинка

index                                                      38649
brand                                                        BMW
0more                                                       2011
1more                                                    лифтбек
2more                                                      серый
3more                                                     Дизель
model          BMW 5 серии Gran Turismo VI (F10/F11/F07) 530d...
год выпуска                                                 2011
Пробег                                                223 390 км
Кузов                                                    лифтбек
Цвет                                                       серый
Двигатель                                     3.0 л / 245 л.с. /
Налог                                             18 375 ₽ / год
Коробка                                           автоматическая
Привод                                                    полный
Руль                     

In [270]:
train_engine=get_unique_values(train_df['engine']) # ОБЪЁМ ДВИГАТЕЛЯ
for entry in train_engine[15:20]:
    print(entry,len(train_engine))

3.5 л / 276 л.с. / 989
1.4 л / 125 л.с. / 989
2.5 л / 199 л.с. / 989
3.3 л / 95 л.с. / 989
3.5 л / 269 л.с. / 989


In [271]:
# Почистим ОБЪЁМ ДВИГАТЕЛЯ, учитывая, что там ещё попадаются ЭЛЕКТРОдвигатели
train_df.loc[:,'engine']=train_df.loc[:,'engine'].apply(lambda x: float(x.split()[0]) if len(x.split())<7 else x.split()[(len(x.split())-1)]) # ОБЪЁМ ДВИГАТЕЛЯ

In [272]:
train_engine=get_unique_values(train_df['engine']) # ЛОШАДИНЫЕ СИЛЫ
print(train_engine,len(train_engine))

[0.7, 1.8, 2.0, 3.4, 2.5, 4.7, 3.3, 3.5, 5.0, 4.5, 5.7, 4.6, 4.0, 4.3, 1.5, 1.0, 3.0, 4.1, 4.4, 4.9, 0.6, 5.5, 5.4, 5.6, 5.9, 6.0, 6.6, 'Электро', 1.6, 2.1, 2.6, 3.6, 3.1, 1.2, 1.7, 2.7, 2.2, 2.8, 2.3, 3.2, 3.7, 3.8, 4.2, 1.3, 4.8, 5.8, 5.2, 6.2, 6.3, 1.4, 1.9, 2.4, 2.9, 3.9] 54


In [273]:
train_df['fuel'].value_counts(dropna=False)  # Проверим, что означают отсутствующие значения

Бензин                                32188
Дизель                                 7573
Гибрид                                  768
Бензин, газобаллонное оборудование      341
NaN                                     152
Газ                                       3
Газ, газобаллонное оборудование           1
Гибрид, газобаллонное оборудование        1
Дизель, газобаллонное оборудование        1
Name: fuel, dtype: int64

In [274]:
display(train_df.groupby(['fuel','engine'],dropna=False).count())

# Что и требовалось доказать, все NAN в топливе - это элеткрокары

brand  model  modeld  body  color  \
fuel                               engine                                       
Бензин                             0.6          1      1       1     1      1   
                                   0.7         59     59      59    59     59   
                                   1.0         70     70      70    70     70   
                                   1.2        273    273     273   273    273   
                                   1.3        479    479     479   479    479   
...                                           ...    ...     ...   ...    ...   
Дизель                             4.2         74     74      74    74     74   
                                   4.5        532    532     532   532    532   
                                   5.9         11     11      11    11     11   
Дизель, газобаллонное оборудование 2.8          1      1       1     1      1   
NaN                                Электро    152    152     152   152    152   

                                            doors  transmiss  drive  steer  \
fuel                               engine                                    
Бензин                             0.6          1          1      1      1   
                                   0.7         59         59     59     59   
                                   1.0         70         70     70     70   
                                   1.2        273        273    273    273   
                                   1.3        479        479    479    479   
...                                           ...        ...    ...    ...   
Дизель                             4.2         74         74     74     74   
                                   4.5        532        532    532    532   
                                   5.9         11         11     11     11   
Дизель, газобаллонное оборудование 2.8          1          1      1      1   
NaN                                Электро    152        152    152    152   

                                            power  year  mileage  owners  \
fuel                               engine                                  
Бензин                             0.6          1     1        1       1   
                                   0.7         59    59       59      59   
                                   1.0         70    70       70      70   
                                   1.2        273   273      273     273   
                                   1.3        479   479      479     479   
...                                           ...   ...      ...     ...   
Дизель                             4.2         74    74       74      74   
                                   4.5        532   532      532     532   
                                   5.9         11    11       11      11   
Дизель, газобаллонное оборудование 2.8          1     1        1       1   
NaN                                Электро    152   152      152     152   

                                            title  price  
fuel                               engine                 
Бензин                             0.6          1      1  
                                   0.7         59     59  
                                   1.0         70     70  
                                   1.2        273    273  
                                   1.3        479    479  
...                                           ...    ...  
Дизель                             4.2         74     74  
                                   4.5        532    532  
                                   5.9         11     11  
Дизель, газобаллонное оборудование 2.8          1      1  
NaN                                Электро    152    152  

[127 rows x 15 columns]

In [275]:
train_df['fuel'].fillna('электро', inplace=True) # Так и запишем

In [276]:
train_df['fuel'].value_counts(dropna=False)

Бензин                                32188
Дизель                                 7573
Гибрид                                  768
Бензин, газобаллонное оборудование      341
электро                                 152
Газ                                       3
Газ, газобаллонное оборудование           1
Гибрид, газобаллонное оборудование        1
Дизель, газобаллонное оборудование        1
Name: fuel, dtype: int64

In [277]:
fuel_type=get_unique_values(train_df['fuel'])
print(fuel_type)

['электро', 'Бензин, газобаллонное оборудование', 'Гибрид, газобаллонное оборудование', 'Гибрид', 'Бензин', 'Газ', 'Дизель, газобаллонное оборудование', 'Дизель', 'Газ, газобаллонное оборудование']


In [278]:
fuel_type=['Бензин, газобаллонное оборудование',  'Гибрид, газобаллонное оборудование', 
           'Дизель, газобаллонное оборудование',  'Газ, газобаллонное оборудование'] # обзовём всех их гибридами

train_df['fuel'] = train_df['fuel'].apply(lambda x: 'гибрид' if x in fuel_type else x.lower())

In [279]:
train_df['fuel'].value_counts(dropna=False)

бензин     32188
дизель      7573
гибрид      1112
электро      152
газ            3
Name: fuel, dtype: int64

In [280]:
#Оставим только численные значения в поле engine:

train_df.loc[:,'engine']=train_df.loc[:,'engine'].apply(lambda x: 0 if x=='Электро' else x)  # fuel

In [281]:
# ТЕСТОВЫЙ

In [282]:
test_engine=(get_unique_values(test_df['engine']))
print(test_engine[:],len(test_engine))

['1.9 LTR', '4.1 LTR', '4.9 LTR', '1.7 LTR', '1.2 LTR', '2.3 LTR', '3.7 LTR', '3.2 LTR', '2.4 LTR', '5.3 LTR', '2.0 LTR', '5.5 LTR', '4.3 LTR', '1.1 LTR', '3.0 LTR', '4.5 LTR', '3.6 LTR', '2.1 LTR', '3.3 LTR', '3.1 LTR', '5.9 LTR', '4.0 LTR', '6.3 LTR', '5.6 LTR', '2.8 LTR', '4.2 LTR', '0.7 LTR', '5.2 LTR', '2.2 LTR', '6.0 LTR', '5.4 LTR', '4.4 LTR', '1.5 LTR', '1.0 LTR', '5.0 LTR', '1.8 LTR', '2.6 LTR', '6.6 LTR', '4.6 LTR', '2.7 LTR', '3.4 LTR', '2.9 LTR', '5.7 LTR', ' LTR', '1.6 LTR', '2.5 LTR', '3.5 LTR', '1.3 LTR', '4.7 LTR', '1.4 LTR', '6.2 LTR', '5.8 LTR', '4.8 LTR', '3.9 LTR', '3.8 LTR'] 55


In [283]:
missing_engine=test_df[test_df['engine']==' LTR'][['engine','brand','model','power','fuel']]  # НЕТ ИНФОРМАЦИИ ОБ ОБЪЁМЕ ДВИГАТЕЛЯ
display(missing_engine)

,engine,brand,model,power,fuel
5241,LTR,AUDI,E,408 N12,электро
7992,LTR,BMW,I3,170 N12,электро
8252,LTR,BMW,I3,170 N12,электро
8350,LTR,BMW,I3,170 N12,электро
8684,LTR,BMW,I3,170 N12,электро
8722,LTR,BMW,I3,170 N12,электро
8917,LTR,BMW,I3,170 N12,электро
10817,LTR,BMW,I3,170 N12,электро
10936,LTR,BMW,I3,170 N12,электро
11043,LTR,BMW,I3,170 N12,электро


In [284]:
test_df['engine']=test_df['engine'].apply(lambda x: 0 if x==' LTR' else float(x.split()[0]) ) # Потому что Электро

In [285]:
display(test_df['engine'].value_counts(bins=5, sort=False))

(-0.0076, 1.32]     1019
(1.32, 2.64]       23340
(2.64, 3.96]        7328
(3.96, 5.28]        2187
(5.28, 6.6]          812
Name: engine, dtype: int64

In [286]:
test_power=(get_unique_values(test_df['power']))  # МОЩНОСТЬ 
print(test_power[11:14],len(test_power))

['71 N12', '101 N12', '330 N12'] 315


In [287]:
test_df['power']=test_df['power'].apply(lambda x: int(x.split()[0]) ) #Уберём информацию об аккумуляторе (N12)

In [288]:
test_power=(get_unique_values(test_df['power']))
print(test_power[11:14],len(test_power))

display(test_df['power'].value_counts(bins=5, sort=False))

[52, 53, 54] 315


(29.389999999999997, 151.8]    14627
(151.8, 273.6]                 15615
(273.6, 395.4]                  3225
(395.4, 517.2]                   771
(517.2, 639.0]                   448
Name: power, dtype: int64

In [289]:
test_fuel=(get_unique_values(test_df['fuel'])) # Всё как надо
print(test_fuel[:],len(test_fuel))

['электро', 'дизель', 'гибрид', 'газ', 'бензин'] 5


### doors   -  to drop

In [290]:
test_doors=(get_unique_values(test_df['doors']))
print(test_doors[:],len(test_doors))

[0, 2, 3, 4, 5] 5


In [291]:
train_doors=get_unique_values(train_df['doors']) # Не смогла найти информацию, чтобы была для всех машин. Появлялась очень нерегулярно
print(train_doors,len(train_doors))   # Будем выбрасывать.

[' '] 1


### transmiss

In [292]:
test_transmiss=(get_unique_values(test_df['transmiss']))
print(test_transmiss[:],len(test_doors))

['вариатор', 'механическая', 'роботизированная', 'автоматическая'] 5


In [293]:
train_transmiss=get_unique_values(train_df['transmiss']) # Боюсь, придётся исключать
print(train_transmiss,len(train_transmiss))

['вариатор', 'механическая', 'роботизированная', 'автоматическая'] 4


### drive

In [294]:
test_drive=(get_unique_values(test_df['drive']))
print(test_drive[:],len(test_drive))

['полный', 'передний', 'задний'] 3


In [295]:
train_drive=(get_unique_values(train_df['drive']))
print(train_drive[:],len(train_drive))

['полный', 'передний', 'задний'] 3


### steer

In [296]:
test_steer=(get_unique_values(test_df['steer']))
print(test_steer[:],len(test_steer))

['Правый', 'Левый'] 2


In [297]:
train_steer=(get_unique_values(train_df['steer']))
print(train_steer[:],len(train_steer))

['Правый', 'Левый'] 2


### year

In [298]:
test_year=(get_unique_values(test_df['year']))
print(test_year[3:7],len(test_year))
print(type(test_year[5]))

display(test_df['year'].value_counts(bins=5, sort=False))

[1939, 1948, 1949, 1950] 69
<class 'int'>


(1903.883, 1927.2]        1
(1927.2, 1950.4]          8
(1950.4, 1973.6]         22
(1973.6, 1996.8]       1871
(1996.8, 2020.0]      32784
Name: year, dtype: int64

In [299]:
test_df[test_df['year']<1955].sort_values('year')

,brand,model,modeld,body,color,engine,doors,transmiss,drive,steer,power,fuel,year,mileage,owners,title
16944,MERCEDES-BENZ,SIMPLEX,1904,кабриолет,белый,5.3,0,механическая,задний,Правый,32,бензин,1904,48000,1 владелец,Оригинал
16891,MERCEDES-BENZ,W136,1936,седан,красный,1.7,4,механическая,задний,Левый,38,бензин,1936,1,1 владелец,Оригинал
9625,BMW,326,1936,седан,бежевый,2.0,4,механическая,задний,Левый,51,бензин,1937,16000,2 владельца,Оригинал
9819,BMW,321,1937,седан 2 дв.,чёрный,2.0,2,механическая,задний,Левый,46,бензин,1937,4500,3 или более,Оригинал
3174,AUDI,920,1938,седан,чёрный,3.3,4,механическая,задний,Левый,75,бензин,1938,90000,2 владельца,Оригинал
1777,SKODA,POPULAR,1934,хэтчбек 3 дв.,чёрный,1.1,3,механическая,задний,Левый,30,бензин,1939,14000,3 или более,Оригинал
33748,BMW,321,1937,седан 2 дв.,коричневый,2.0,2,механическая,задний,Левый,46,бензин,1948,10,3 или более,Оригинал
9495,BMW,321,1937,седан 2 дв.,чёрный,2.0,2,механическая,задний,Левый,46,бензин,1949,25000,1 владелец,Оригинал
9143,BMW,340,1949,седан,чёрный,2.0,4,механическая,задний,Левый,51,бензин,1950,44100,3 или более,Оригинал
18185,MERCEDES-BENZ,W188,1951,купе,золотистый,3.0,2,механическая,задний,Левый,150,бензин,1952,45200,1 владелец,Оригинал


In [300]:
# В тренинговом сете нет антикварных машин, это грустно
# Я не нашла предложений по таким машинам даже на сторонних сайтах

train_year=(get_unique_values(train_df['year']))
print(train_year[5:10],len(train_year))
print(type(train_year[5]))

display(train_df['year'].value_counts(bins=5, sort=False))  

[1965.0, 1968.0, 1969.0, 1972.0, 1974.0] 57
<class 'float'>


(1937.916, 1954.6]        3
(1954.6, 1971.2]         45
(1971.2, 1987.8]        262
(1987.8, 2004.4]       5942
(2004.4, 2021.0]      34776
Name: year, dtype: int64

### mileage

In [301]:
test_mileage=(get_unique_values(test_df['mileage']))
print(test_mileage[555:557],len(test_mileage))
print(type(test_mileage[5]))

display(test_df['mileage'].value_counts(bins=5, sort=False))

[165152, 66848] 11268
<class 'int'>


(-999.0, 200000.8]       24773
(200000.8, 400000.6]      9219
(400000.6, 600000.4]       635
(600000.4, 800000.2]        30
(800000.2, 1000000.0]       29
Name: mileage, dtype: int64

In [302]:
train_mileage=(get_unique_values(train_df['mileage']))
print(train_mileage[8765:8767],len(train_mileage))
print(type(train_mileage[5]))


['60 182 км', '138 482 км'] 13623
<class 'str'>


In [303]:
train_df.loc[:,'mileage']=train_df.loc[:,'mileage'].apply(lambda x: int(''.join(x.split()[0:2])) if len(x.split())>2 else int(x.split()[0]) )

In [304]:
display(train_df['mileage'].value_counts(bins=5, sort=False))

(-998.999, 200000.6]    32943
(200000.6, 400000.2]     7614
(400000.2, 599999.8]      437
(599999.8, 799999.4]       24
(799999.4, 999999.0]       10
Name: mileage, dtype: int64

### owners

In [305]:
test_owners=(get_unique_values(test_df['owners']))
print(test_owners[:],len(test_owners))

['2\xa0владельца', '1\xa0владелец', '3 или более'] 3


In [306]:
train_owners=(get_unique_values(train_df['owners']))
print(train_owners[:],len(train_owners))

['1 владелец', '2 владельца', '3 или более'] 3


In [308]:
test_df['owners']=test_df['owners'].apply(lambda x: int(re.compile(r'\b\d').match(x).group())) # Почистим ...

In [309]:
test_owners=(get_unique_values(test_df['owners']))
print(test_owners[:],len(test_owners))

[1, 2, 3] 3


In [310]:
train_df['owners']=train_df['owners'].apply(lambda x: int(re.compile(r'\b\d').match(x).group())) # ...и превратим в integer

In [311]:
train_owners=(get_unique_values(train_df['owners']))
print(train_owners[:],len(train_owners))

[1, 2, 3] 3


### title

In [312]:
test_title=(get_unique_values(test_df['title']))
print(test_title[:],len(test_title))

['Оригинал', 'Дубликат'] 2


In [313]:
train_title=(get_unique_values(train_df['title']))
print(train_title[:],len(train_title))

['Оригинал', 'Дубликат'] 2


### price

In [314]:
train_price=(get_unique_values(train_df['price']))
print(type(train_price[2]),len(train_price))

<class 'str'> 4667


In [315]:
# Некоторые машины слишком быстро продаются...

train_df['price']=train_df['price'].apply(lambda x: 0 if x=='SOLD' else int(x.strip()))

In [316]:
display(train_df['price'].value_counts(bins=5, sort=False))

(-40000.001, 8000000.0]     40415
(8000000.0, 16000000.0]       537
(16000000.0, 24000000.0]       67
(24000000.0, 32000000.0]        8
(32000000.0, 40000000.0]        1
Name: price, dtype: int64

In [317]:
train_df[train_df['price']<=0]

,brand,model,modeld,body,color,engine,doors,transmiss,drive,steer,power,fuel,year,mileage,owners,title,price
18,LEXUS,LX,,внедорожник 5 дв.,серый,5.7,,автоматическая,полный,Левый,367,бензин,2015.0,78000,2,Оригинал,0
47,LEXUS,LX,,внедорожник 5 дв.,белый,5.7,,автоматическая,полный,Левый,367,бензин,2015.0,105000,3,Дубликат,0
99,LEXUS,GX,,внедорожник 5 дв.,коричневый,4.6,,автоматическая,полный,Левый,296,бензин,2013.0,114000,3,Оригинал,0
126,LEXUS,GS,,седан,чёрный,3.5,,автоматическая,полный,Левый,317,бензин,2015.0,140000,2,Оригинал,0
164,LEXUS,GX,,внедорожник 5 дв.,белый,4.7,,автоматическая,полный,Левый,270,бензин,2005.0,237384,1,Оригинал,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39044,BMW,7,,седан,чёрный,3.0,,автоматическая,полный,Левый,249,дизель,2019.0,12300,1,Оригинал,0
39065,BMW,X5,,внедорожник 5 дв.,белый,3.0,,автоматическая,полный,Левый,306,бензин,2013.0,147000,3,Оригинал,0
39083,BMW,7,,седан,чёрный,3.0,,автоматическая,полный,Левый,249,дизель,2019.0,12300,1,Оригинал,0
39104,BMW,X5,,внедорожник 5 дв.,белый,3.0,,автоматическая,полный,Левый,306,бензин,2013.0,147000,3,Оригинал,0


In [318]:
train_df.drop(train_df[train_df['price']<=0].index, inplace = True)

In [319]:
display(train_df['price'].value_counts(bins=5, sort=False))

(-14975.001, 8020000.0]     39655
(8020000.0, 16015000.0]       537
(16015000.0, 24010000.0]       67
(24010000.0, 32005000.0]        8
(32005000.0, 40000000.0]        1
Name: price, dtype: int64

In [320]:
train_df[train_df['price']>25000000]

,brand,model,modeld,body,color,engine,doors,transmiss,drive,steer,power,fuel,year,mileage,owners,title,price
19037,MERCEDES-BENZ,V,,минивэн,чёрный,2.1,,автоматическая,полный,Левый,190,дизель,2020.0,100,1,Оригинал,27800000
19038,MERCEDES-BENZ,V,,минивэн,чёрный,2.1,,автоматическая,полный,Левый,190,дизель,2020.0,1,1,Оригинал,25600000
19208,MERCEDES-BENZ,V,,минивэн,чёрный,2.1,,автоматическая,полный,Левый,190,дизель,2020.0,32,1,Оригинал,27000000
20463,MERCEDES-BENZ,G,,внедорожник открытый,голубой,5.0,,автоматическая,полный,Левый,296,бензин,2008.0,1341,1,Оригинал,31500000
21013,MERCEDES-BENZ,G,,внедорожник 5 дв.,синий,4.0,,автоматическая,полный,Левый,585,бензин,2020.0,4000,1,Оригинал,27500000
21408,MERCEDES-BENZ,SLS,,родстер,красный,6.2,,роботизированная,задний,Левый,591,бензин,2014.0,4800,3,Оригинал,26000000
21644,MERCEDES-BENZ,G,,внедорожник 5 дв.,чёрный,5.5,,автоматическая,полный,Левый,544,бензин,2014.0,13200,2,Оригинал,40000000
22048,MERCEDES-BENZ,SLS,,родстер,серый,6.2,,роботизированная,задний,Левый,591,бензин,2014.0,12535,1,Оригинал,28000000


# # Drop modeld & doors

In [321]:
train_df.drop(['modeld', 'doors'], axis=1,inplace=True)

In [322]:
test_df.drop(['modeld', 'doors'], axis=1,inplace=True)

# Продолжение следует...

In [323]:
train_df.to_csv('train_df_cleaned.csv',index=False)  # Save the dataframe

In [324]:
test_df.to_csv('test_df_cleaned.csv',index=False)  # Save the dataframe